# Schematics 4.0 — FaceStats
Updated diagrams reflecting the current v4 repo layout and data flow.

## Tools

```mermaid
flowchart LR
    subgraph PRE[Preprocessing]
        MP[Mediapipe FaceMesh]
        CV2[OpenCV]
        PIL[Pillow]
        TQDM[tqdm]
    end
    subgraph EMB[Embeddings]
        HF[HuggingFace Transformers]
        TORCH[PyTorch]
    end
    subgraph ATTR[Attributes]
        HFATTR[HF Pipelines]
    end
    subgraph DATA[Data]
        PL[Polars]
        NP[NumPy]
    end
    subgraph VIZ[Visualization]
        MPL[Matplotlib/Seaborn]
    end

    PRE --> EMB
    EMB --> ATTR
    EMB --> DATA
    ATTR --> DATA
    DATA --> VIZ
```


## Diagram 1 — FaceStats v4 High Level Pipeline

```mermaid
flowchart LR
    A["📁 Raw Images
 data/raw/"] --> B["🧹 Preprocess
 src/pipeline/preprocess.py -> data/processed/preproc/"]
    B --> C["🧠 CLIP Embeddings
 notebooks/02_embeddings.ipynb -> data/processed/embeddings_clip.parquet"]
    C --> D["👥 Attributes
 notebooks/03_attributes.ipynb -> data/processed/metadata/attributes.parquet"]
    C --> E["💚 Attractiveness Train
 notebooks/04_attractiveness_model.ipynb -> models/attractiveness_regressor.pt"]
    C --> F["💚 Attractiveness Inference
 notebooks/05_attractiveness_inference.ipynb -> data/processed/metadata/attractiveness_scores.parquet"]
    D --> G["📊 Clean/Flags
 attributes_clean.parquet / attributes_flags.parquet"]
    D --> H["🌍 Clusters
 notebooks/05_ethnicity_clusters.ipynb -> attributes_with_clusters.parquet"]
    F --> I["🔗 Join Scores
 downstream merge with attributes_with_meta"]
    G --> I
    H --> I
```


## Diagram 2 — Preprocessing Flow

```mermaid
flowchart LR
    A["📁 data/raw/"] --> B["🔍 Load
Pillow"]
    B --> C["🧽 RGB + Normalize"]
    C --> D["📏 Resize 512x512"]
    D --> E["💾 Save JPG
 data/processed/preproc/"]
```


## Diagram 3 — Embedding Extraction

```mermaid
flowchart LR
    A["🖼️ Preprocessed
 data/processed/preproc/"] --> B["📥 Tokenize
 CLIPProcessor"]
    B --> C["🧠 Forward
 CLIP/ViT (torch + HF)"]
    C --> D["📏 L2 Normalize
 512-D"]
    D --> E["💾 Parquet
 data/processed/embeddings_clip.parquet"]
```


## Diagram 4 — Attributes

```mermaid
flowchart LR
    A["🧠 Embeddings + Images"] --> B["👤 Age/Gender/Ethnicity
 HF pipelines"]
    B --> C["📄 attributes.parquet"]
    C --> D["✅ attributes_clean.parquet"]
    C --> E["🚩 attributes_flags.parquet"]
    C --> F["🌍 attributes_with_clusters.parquet"]
```

## Diagram 5 — Attractiveness

```mermaid
flowchart LR
    A["🔢 Embeddings
 data/processed/embeddings_clip.parquet"] --> B["🧠 Train MLP
 notebooks/04_attractiveness_model.ipynb"]
    B --> C["💾 Model
 models/attractiveness_regressor.pt"]
    A --> D["🏃 Inference
 notebooks/05_attractiveness_inference.ipynb"]
    C --> D
    D --> E["📄 Scores
 data/processed/metadata/attractiveness_scores.parquet"]
```

## Diagram 6 — Repo Map (Current)

```text
FaceStats/
├── LICENSE
├── README.md (v3.5), readme2.md (v4)
├── requirements.txt
├── schematics.ipynb
├── schematics_4_0.ipynb (this file)
├── tools_summary.md, repo_map.md, files_and_folders.md
├── x.ipynb, del.ipynb
├── data/
│   ├── processed/
│   │   ├── embeddings_clip.parquet
│   │   ├── preproc/ (resized images)
│   │   └── metadata/
│   │       ├── attributes.parquet
│   │       ├── attributes_clean.parquet
│   │       ├── attributes_flags.parquet
│   │       ├── attributes_with_clusters.parquet
│   │       ├── attributes_with_meta.parquet
│   │       └── (attractiveness_scores.parquet expected after inference)
│   └── raw/ (sample images)
├── models_insightface/ (weights/configs)
├── models/ (created by training; holds attractiveness_regressor.pt)
├── notebooks/
│   ├── 01_preprocess.ipynb
│   ├── 02_embeddings.ipynb
│   ├── 03_attributes.ipynb
│   ├── 04_visualize_attributes.ipynb
│   ├── 04_attractiveness_model.ipynb
│   ├── 05_attractiveness_inference.ipynb
│   ├── 05_ethnicity_clusters.ipynb (and copy)
│   └── (TODO) composites, additional inference notebooks
├── src/
│   ├── pipeline/preprocess.py
│   ├── embeddings/embed_clip.py
│   ├── attributes/face_attributes.py, age_infer.py
│   ├── models/attractiveness_model.py, train_attractiveness.py
│   ├── composite/ (generators)
│   ├── metadata/build_master.py
│   ├── data_utils/, visualization/, learning/
└── config/
```
